In [ ]:
pip install --upgrade pip

In [ ]:
pip install pandas geopandas geopy shapely openpyxl censusgeocode

In [ ]:
import pandas as pd
import geopandas as gpd
#from censusgeocode import CensusGeocode
from shapely.geometry import Point
import time

df = pd.read_excel('LWV_members.xlsx')
df.head()

In [ ]:
import pandas as pd
import requests
import time

def geocode_census_api(address, zip_code):
    """Geocode using Census API directly"""
    url = "https://geocoding.geo.census.gov/geocoder/locations/onelineaddress"
    params = {
        'address': f"{address}, {zip_code}",
        'benchmark': 'Public_AR_Current',
        'format': 'json'
    }
    
    try:
        response = requests.get(url, params=params, timeout=10)
        data = response.json()
        
        if data['result']['addressMatches']:
            coords = data['result']['addressMatches'][0]['coordinates']
            return coords['y'], coords['x']  # lat, lon
        return None, None
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
        return None, None


# Geocode each address
results = []
for idx, row in df.iterrows():
    if idx % 10 == 0:
        print(f"Processing {idx}/{len(df)}...")
    
    lat, lon = geocode_census_api(row['address1'], row['zip'])
    results.append({'latitude': lat, 'longitude': lon})
    
    time.sleep(0.1)  # Be polite to the API

# Add coordinates to dataframe
df[['latitude', 'longitude']] = pd.DataFrame(results)

# Continue with geopandas spatial join
import geopandas as gpd
from shapely.geometry import Point

# Remove rows that couldn't be geocoded
df_valid = df.dropna(subset=['latitude', 'longitude'])

# Convert to GeoDataFrame
geometry = [Point(xy) for xy in zip(df_valid['longitude'], df_valid['latitude'])]
gdf = gpd.GeoDataFrame(df_valid, geometry=geometry, crs='EPSG:4326')# Read districts and perform spatial join
districts = gpd.read_file('Waukesha_geo/Supervisory_Districts_Adopted_11_9_21.shp')
#districts = gpd.read_file('MKE_county_wards/MilwaukeeCounty_VotingWards.shp')
gdf = gdf.to_crs(districts.crs)
result = gpd.sjoin(gdf, districts, how='left', predicate='within')

# Save results
result.to_excel('addresses_with_districts_wc.xlsx', index=False)


In [ ]:
result.head()

In [ ]:
nulldf = result[result['DISTRICT'].isnull()]
nulldf

In [ ]:
result.shape

In [ ]:
#mke_df = result[result['Muni'].notnull()]
#mke_df.shape
wauk_df = result[result['DISTRICT'].notnull()]
wauk_df.shape

In [ ]:
#mke_df.rename(columns={'index_right': 'index_right_original', 'geometry': 'geometry_addr'}, inplace=True)
#mke_df.head()
wauk_df.rename(columns={'index_right': 'index_right_original', 'geometry': 'geometry_addr'}, inplace=True)
wauk_df.head()

In [ ]:
#mke_df['Muni'] = mke_df['Muni'].astype(int)
wauk_df['DISTRICT'] = wauk_df['DISTRICT'].astype(int)

In [ ]:
wauk_df.to_excel('waukesha_addresses_with_districts.xlsx', index=False)

In [ ]:
rep_map_df = gpd.read_file('MKE_rep_geo/City_Common_Council_District_or_Village_Board_of_Trustees_in_Milwaukee_County.shp', )
rep_map_df.head()

In [ ]:
rep_map_df.rename(columns = {'Member_Nam': 'incumbent'}, inplace=True)
rep_map_df.columns

In [ ]:
print(rep_map_df.crs)
print(mke_df.crs)

In [ ]:

# --- Inputs ---
districts_gdf = rep_map_df.copy()   # municipal polygons, already EPSG:6609 per your print
addresses_any = mke_df.set_geometry('geometry_addr')           # either GeoDataFrame with geometry, or DataFrame with lat/lon

# --- Confirm CRS ---
print("Districts CRS:", districts_gdf.crs)  # should be EPSG:6609
print("Addresses type:", type(addresses_any))
print("Addresses CRS:", getattr(addresses_any, "crs", None))

# --- Build addresses GeoDataFrame correctly ---
if isinstance(addresses_any, gpd.GeoDataFrame) and addresses_any.geometry is not None:
    # Case A: already a GeoDataFrame
    addresses_gdf = addresses_any.copy()
    if addresses_gdf.crs is None:
        raise ValueError("addresses_gdf has geometry but no CRS; set the correct EPSG (6609 or 4326).")
    # If addresses are not in 6609, reproject to 6609
    if addresses_gdf.crs.to_epsg() != 6609:
        addresses_gdf = addresses_gdf.to_crs("EPSG:6609")
else:
    # Case B: DataFrame with lat/lon (in degrees)
    # 1) create points in EPSG:4326
    addresses_gdf = gpd.GeoDataFrame(
        addresses_any.copy(),
        geometry=gpd.points_from_xy(addresses_any["longitude"], addresses_any["latitude"]),
        crs="EPSG:4326"
    )
    # 2) reproject points to EPSG:6609 to match districts
    addresses_gdf = addresses_gdf.to_crs("EPSG:6609")

# --- Ensure districts_gdf CRS is set (it is, but guard anyway) ---
if districts_gdf.crs is None:
    raise ValueError("districts_gdf.crs is None—set it to 'EPSG:6609' (or the known CRS).")

# --- Sanity check: bounds should overlap ---
print("Addresses bounds (6609):", addresses_gdf.total_bounds)
print("Districts bounds (6609):", districts_gdf.total_bounds)

# --- Spatial join: point-in-polygon ---
joined = gpd.sjoin(
    addresses_gdf,
    districts_gdf,
    how="left",
    predicate="within",
    lsuffix="_addr",
    rsuffix="_poly"
)

# --- Verify the district column exists on the right frame ---
print("District columns in polygons:", [c for c in districts_gdf.columns if c != "geometry"])
assert "District_D" in districts_gdf.columns, "Column 'District_D' not found in districts_gdf."

# --- Columns to keep (only keep those that exist to avoid KeyError) ---
desired_cols = [
    'last_name', 'first_name', 'address1', 'address2', 'city', 'state',
    'zip', 'primary_email', 'membership_status_description',
    'Ward_ID', 'AlderID', 'SuperID', 'Muni', 'latitude', 'longitude',
    'District_D', 'incumbent', 'geometry'  # geometry of the address point in EPSG:6609
]
cols_to_keep = [c for c in desired_cols if c in joined.columns]
output = joined[cols_to_keep]

output.head()


In [ ]:
output['District_D'].isnull().sum()

In [ ]:
print(output.shape)
output = output.drop_duplicates(subset=["latitude", "longitude"], keep="first")
print(output.shape)

In [ ]:

# 1) First word (e.g., "City", "Town", "Village")
# - Strip leading/trailing whitespace, then extract the first alphabetic token.
output["Muni_type"] = (
    output["District_D"]
        .astype(str)
        .str.strip()
        .str.extract(r'^([A-Za-z-]+)', expand=False)  # allow hyphenated like "Town-Village" if needed
)


last_parts = (
    output["District_D"]
        .astype(str)
        .str.strip()
        .str.extract(r'([A-Za-z-]+)(?:\s+(\d+))?$', expand=True)  # small tweak to require space before digits
)

# Build Rep_type: "Word N" if N exists, else "Word"
output["Rep_type"] = last_parts[0].str.cat(last_parts[1], sep=" ", na_rep="")


# Quick preview
output[["District_D", "Muni_type", "Rep_type"]].head()


In [ ]:
output.columns

In [ ]:
output.drop(columns = 'last_word').to_excel('mke_county_list.xlsx', index=False)